# Mcp Protocol Messages

## Ping

In [1]:
curl -s -D - -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "ping"
}'

HTTP/1.1 200 OK
Content-Type: text/event-stream
Date: Fri, 01 Aug 2025 00:32:46 GMT
Server: Kestrel
Cache-Control: no-cache,no-store
Content-Encoding: identity
Transfer-Encoding: chunked
Mcp-Session-Id: qE1hDeh6qFz7MfBhM3DG2w

event: message
data: {"result":{},"id":1,"jsonrpc":"2.0"}



## Initialize

In [ ]:
curl -s -D - -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "clientInfo": {
            "name": "RestClient",
            "version": "0.1.0"
        },
        "capabilities": {},
        "protocolVersion": "{{ProtocolVersion}}"
    }
}'

In [ ]:
# Save the Mcp-Session ID from the response for subsequent requests
#!set --name McpSessionId --value GLh-x6CIRbV6icKmSQiosw

## Tools List

In [ ]:
curl -s -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-H "Mcp-Session-Id: $McpSessionId" `
-d '{
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list"
}' | Where-Object { $_ -like "data:*" } | ForEach-Object { $_.Substring(5) } | jq

## Tool Call

In [ ]:
curl -s -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-H "Mcp-Session-Id: $McpSessionId" `
-d '{
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "add",
        "arguments": {
            "a": 5,
            "b": 3
        }
    }
}' | Where-Object { $_ -like "data:*" } | ForEach-Object { $_.Substring(5) } | jq

In [ ]:
curl -s -D - -X GET http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Mcp-Session-Id: $McpSessionId"

In [ ]:
#nullable enable
// C# code to make an http request to call the `get_products` tool
// And handle the SSE response
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;
using System.Text.Json;
using System.IO;

// Send an HTTP POST request to http://localhost:3001
// and handle the Server-Sent Events (SSE) response.
var httpClient = new HttpClient();

// Set the standard headers for requests to the MCP server
httpClient.DefaultRequestHeaders.Add("MCP-Protocol-Version", "2025-06-18");
httpClient.DefaultRequestHeaders.Add("Accept", "application/json, text/event-stream");
//httpClient.DefaultRequestHeaders.Add("Content-Type", "application/json");

// The request body should be a JSON RPC request to call the `LongRunningOperation` tool
var requestBody = new
{
    jsonrpc = "2.0",
    id = 2,
    method = "tools/call",
    @params = new
    {
        _meta = new
        {
            progressToken = "abc123"
        },
        name = "longRunningOperation"
    }
};
var response = await httpClient.PostAsync("http://localhost:3001/", new StringContent(JsonSerializer.Serialize(requestBody), Encoding.UTF8, "application/json"));
response.EnsureSuccessStatusCode();

var reader = new StreamReader(await response.Content.ReadAsStreamAsync());
while (!reader.EndOfStream)
{
    var line = await reader.ReadLineAsync();
    Console.WriteLine(line);
}


In [ ]:
// Send an HTTP GET request to http://localhost:3001
// and handle the Server-Sent Events (SSE) response.
var httpClient = new HttpClient();

// Set the standard headers for requests to the MCP server
httpClient.DefaultRequestHeaders.Add("MCP-Protocol-Version", "2025-06-18");
httpClient.DefaultRequestHeaders.Add("Accept", "application/json, text/event-stream");

var response = await httpClient.GetAsync("http://localhost:3001/");
response.EnsureSuccessStatusCode();

var reader = new StreamReader(await response.Content.ReadAsStreamAsync());
while (!reader.EndOfStream)
{
    var line = await reader.ReadLineAsync();
    Console.WriteLine(line);
}